In [1]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# model_name = "allenai/unifiedqa-t5-large"  # you can specify the model size here
# tokenizer = T5Tokenizer.from_pretrained(model_name)

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "allenai/unifiedqa-v2-t5-large-1363200"  # you can specify the model size here
tokenizer = T5Tokenizer.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [2]:
model = T5ForConditionalGeneration.from_pretrained(model_name,
                                                   device_map='auto')

In [6]:
QUESTION_PAD = 45
MAX_ANSWER_LENGTH = 40

import itertools

perm_order = list(itertools.permutations([0, 1, 2, 3], 4))

import textwrap
import numpy as np
from tqdm import trange

import pandas as pd


def measure_unalike(arr, print_arr=False):
    n = len(arr)
    arr = pd.Series(arr).value_counts()
    if print_arr:
        print(arr)
    return 1 - ((arr / n)**2).sum()


question_to_do = 5
per_question = 20


In [7]:
import pickle

dataset_test = pickle.load(open('test_without_abcd.pkl', 'rb'))
dataset_test[0]

('A person wants to start saving money so that they can afford a nice vacation at the end of the year. After looking over their budget and expenses, they decide the best way to save money is to \\n ( ) make more phone calls ( ) quit eating lunch out ( ) buy less with monopoly money ( ) have lunch with friends',
 'quit eating lunch out')

In [9]:
def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    print(input_ids)
    # print(torch.argwhere(input_ids[0]==2)[0,0]+2)
    res = model.generate(input_ids.to(0),
                         **generator_args,
                         max_new_tokens=MAX_ANSWER_LENGTH)
    return tokenizer.batch_decode(res, skip_special_tokens=True)
run_model(dataset_test[0][0])

tensor([[   71,   568,  2746,    12,   456,  4380,   540,    78,    24,    79,
            54,  5293,     3,     9,  1245,  4257,    44,     8,   414,    13,
             8,   215,     5,   621,   479,   147,    70,  1487,    11,  5159,
             6,    79,  2204,     8,   200,   194,    12,  1097,   540,    19,
            12,     3,     2,    29,    41,     3,    61,   143,    72,   951,
          3088,    41,     3,    61, 10399,  3182,  3074,    91,    41,     3,
            61,   805,   705,    28,     3, 23507,    63,   540,    41,     3,
            61,    43,  3074,    28,   803,     1]])


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
def get_prompt(index, perm):
    json_line = dataset["test"][index]
    question = json_line["question_stem"]
    choices = json_line["choices"]
    choice_texts = choices["text"]
    choice_texts = [choice_texts[perm[i]] for i in range(len(choice_texts))]

    def change(text, leng=20, is_question=False):
        pad_x = [0] * leng
        encoded = tokenizer.encode(text)[:-1]
        if len(encoded) > leng:
            print('too long ', len(encoded), leng, is_question)
        pad_x[:len(encoded)] = encoded
        return tokenizer.decode(pad_x)

    candidates = " ".join([
        change(f'( ) {text}', ANSWER_PAD)
        for text, label in zip(choice_texts, choices["label"])
    ]).replace("\n", " ")
    # print(json_line)
    answer_key = json_line["answerKey"]
    answer_key_idx = ord(answer_key[0]) - ord("A")
    # answer_text = choice_texts[answer_key_idx]
    # id = "OBQA_" + json_line['id']
    question_pad = f'{question} \\n '
    prompt = f"{change(question_pad,QUESTION_PAD, is_question=True)}{candidates}"
    return prompt, choice_texts


In [7]:
# answer=[[]]*question_to_do

for i in range(question_to_do):
    choice0 = None
    answer = []
    for k in trange(per_question):
        prompt, choice = get_prompt(i, perm_order[k % 24])
        if choice0 is None:
            choice0 = choice
        # print(perm, textwrap.fill(prompt))
        answer.append(run_model(prompt)[0])
    print('question ', i, choice0, measure_unalike(answer, print_arr=True))

# for index,ans in enumerate(answer):

pass

100%|██████████| 20/20 [00:06<00:00,  3.28it/s]


quit eating lunch with friends    20
Name: count, dtype: int64
question  0 ['make more phone calls', 'quit eating lunch out', 'buy less with monopoly money', 'have lunch with friends'] 0.0


100%|██████████| 20/20 [00:05<00:00,  3.97it/s]


a deserth    20
Name: count, dtype: int64
question  1 ['a marsh', 'a tundra', 'the plains', 'a desert'] 0.0


100%|██████████| 20/20 [00:04<00:00,  4.16it/s]


grasslions    20
Name: count, dtype: int64
question  2 ['lions', 'humans', 'bunnies', 'grass'] 0.0


100%|██████████| 20/20 [00:05<00:00,  3.44it/s]


parts may fall apart    20
Name: count, dtype: int64
question  3 ['roots may be split', 'roots may begin to die', 'parts may break the concrete', 'roots may fall apart'] 0.0


100%|██████████| 20/20 [00:04<00:00,  4.06it/s]

fuel conductors    20
Name: count, dtype: int64
question  4 ['gasoline', 'a power station', 'electrical conductors', 'fuel'] 0.0
